### Setup your environnement variables

In [5]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

# Get started

In [6]:
from dotenv import load_dotenv
assert load_dotenv(override=True) # Load environment variables from a .env file

In [7]:
from uiform import AsyncUiForm
uiclient = AsyncUiForm()
await uiclient.models.list()

['claude-3-5-sonnet-latest',
 'claude-3-5-sonnet-20241022',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'claude-3-haiku-20240307',
 'gpt-4o',
 'gpt-4o-mini',
 'chatgpt-4o-latest',
 'gpt-4o-2024-11-20',
 'gpt-4o-2024-08-06',
 'gpt-4o-2024-05-13',
 'gpt-4o-mini-2024-07-18',
 'o1-2024-12-17',
 'o1-mini-2024-09-12',
 'grok-2-vision-1212',
 'grok-2-1212',
 'gemini-2.0-flash-exp',
 'gemini-1.5-flash-8b',
 'gemini-1.5-flash',
 'gemini-1.5-pro']

# Example 1 - Analyze informations from an email

#### Option A - You use our preprocess endpoint and openAI client

In [8]:
import json
from uiform import AsyncUiForm, Schema
from openai import AsyncOpenAI

with open("freight/schema.json", "r") as f:
    json_schema = json.load(f)


uiclient = AsyncUiForm()
doc_msg = await uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
)

schema_obj = Schema(
    json_schema = json_schema
)


# Now you can use your favorite model to analyze your document
client = AsyncOpenAI()
client.beta.chat.completions.stream
completion = await client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    }
)

# Validate the response against the original schema if you want to remove the reasoning fields
assert completion.choices[0].message.content is not None
extraction = schema_obj.pydantic_model.model_validate_json(
    completion.choices[0].message.content
)

print(extraction)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number='GB123456789', city='London', postal_code='WC2N 5DU', country='GB', code=None, email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='London', postal_code='WC2N 5DU', country='GB', line1='123 Elm Street', line2='Suite 500'), phone_number='+44 20 7946 0958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations='Standard loading procedures apply.'), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+49 89 123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time=

#### Option B - Using UiForm `extractions.parse` endpoint

In [9]:
import json
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()
completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
)

print(completion.choices[0].message.parsed)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number='GB123456789', city='London', postal_code='WC2N 5DU', country='GB', code='SHIP-001', email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='London', postal_code='WC2N 5DU', country='GB', line1='123 Elm Street', line2='Suite 500'), phone_number='+44 20 7946 0958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations=None), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+49 89 123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time='16:00:00'), observations=

#### Option C - Using UiForm `extractions.stream` endpoint

In [10]:
import json
from uiform.client import AsyncUiForm
from IPython.display import clear_output, display

uiclient = AsyncUiForm()

async with uiclient.documents.extractions.stream(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
) as stream:
    async for completion in stream:
        if completion.choices[0].message.parsed is not None:
            clear_output(wait=True)
            display(completion.choices[0].message.parsed.model_dump())

{'booking_id': 'SHIP-001',
 'payment': {'total_price': 1500.0, 'currency': 'EUR'},
 'client': {'company_name': 'ACME Corporation',
  'VAT_number': 'GB123456789',
  'city': 'London',
  'postal_code': 'WC2N 5DU',
  'country': 'GB',
  'code': 'SHIP-001',
  'email': 'client@acme.com'},
 'shipments': [{'shipment_id': 'SHIP-001',
   'sender': {'company_name': 'ACME Corporation',
    'address': {'city': 'London',
     'postal_code': 'WC2N 5DU',
     'country': 'GB',
     'line1': '123 Elm Street',
     'line2': 'Suite 500'},
    'phone_number': '+44 20 7946 0958',
    'email_address': 'client@acme.com',
    'pickup_datetime': {'date': '2023-02-05',
     'start_time': '08:00:00',
     'end_time': '12:00:00'},
    'observations': None},
   'recipient': {'company_name': 'Beta Industries',
    'address': {'city': 'Munich',
     'postal_code': '80331',
     'country': 'DE',
     'line1': '789 Pine Street',
     'line2': None},
    'phone_number': '+49 89 123456',
    'email_address': None,
    'de

## Appendix A - Use text as modality

In [ ]:
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()

completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document="freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    modality='text'
)

print(completion.choices[0].message.parsed)

## Appendix B - Add additional context

In [ ]:
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()

text_operations = {'regex_instructions' : [
        {
            "name": "vat_number",
            "pattern": r"[Ff][Rr]\s*(\d\s*){11}",
            "description": "VAT number in the format XX999999999"
        }
    ]
}

completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    text_operations=text_operations
)

print(completion.choices[0].message.parsed)